# setting_grid 파일의 목표
1. 인프라 데이터 프레임 위,경도 검토 및 좌표 시각화
- 앞서 제작한 인프라 데이터의 위,경도 데이터를 시각화 함으로써 정상적인 데이터인지 확인
2. 격자화
- raw 데이터의 위 경도 데이터를 격자화 시켜, 각 격자 별 데이터로서 구분할 수 있도록 함
3. 병합
- meta 데이터프레임의 생성을 통해 기존 계획한 데이터 프레임의 형태로 제작될 수 있는지 확인.

# 인프라 데이터 프레임 위,경도 검토 및 좌표 시각화

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('인프라_3개제외.csv')

In [3]:
import folium
df[['폐업일자','영업상태명']] = df[['폐업일자','영업상태명']].fillna(0)
test = df.dropna()
test

,인허가일자,좌표정보(X),좌표정보(Y),폐업일자,영업상태명,정보
0,1992-07-22,127.091354,37.538181,19990330,폐업,서울시광진구골프연습장업인허가정보.csv
1,1993-02-27,127.106577,37.547179,19980703,폐업,서울시광진구골프연습장업인허가정보.csv
2,1993-05-18,127.091249,37.537586,20230130,폐업,서울시광진구골프연습장업인허가정보.csv
3,1994-01-27,127.081989,37.555756,20080422,폐업,서울시광진구골프연습장업인허가정보.csv
4,1995-01-14,127.095874,37.538627,0,영업/정상,서울시광진구골프연습장업인허가정보.csv
...,...,...,...,...,...,...
30497,2023-10-25,127.066891,37.535869,0,영업/정상,서울시광진구휴게음식점인허가정보.csv
30498,2023-10-25,127.061296,37.536253,0,영업/정상,서울시광진구휴게음식점인허가정보.csv
30499,2023-10-27,127.085362,37.557365,0,영업/정상,서울시광진구휴게음식점인허가정보.csv
30500,2023-11-02,127.078977,37.555599,0,영업/정상,서울시광진구휴게음식점인허가정보.csv


In [4]:
a = []
for i in range( test.shape[0]):
    arr =[test.iloc[i][1],test.iloc[i][2],test.iloc[i][-1]]
    a.append(arr)

In [5]:
m = folium.Map(location=[37.555, 127.081], zoom_start=13)

In [6]:
def set_folium(x,y,z):
    folium.Marker([x,y],popup=z).add_to(m)
for i in a[::100]:
    set_folium(i[1],i[0],i[2])

In [7]:
m.save('index.html')  # HTML 파일로 저장합니다.


- 정상적으로 인프라 좌표가 생성된 것 확인
---

# 격자화

## 1. 인프라 데이터 프레임 격자화

In [8]:
test

,인허가일자,좌표정보(X),좌표정보(Y),폐업일자,영업상태명,정보
0,1992-07-22,127.091354,37.538181,19990330,폐업,서울시광진구골프연습장업인허가정보.csv
1,1993-02-27,127.106577,37.547179,19980703,폐업,서울시광진구골프연습장업인허가정보.csv
2,1993-05-18,127.091249,37.537586,20230130,폐업,서울시광진구골프연습장업인허가정보.csv
3,1994-01-27,127.081989,37.555756,20080422,폐업,서울시광진구골프연습장업인허가정보.csv
4,1995-01-14,127.095874,37.538627,0,영업/정상,서울시광진구골프연습장업인허가정보.csv
...,...,...,...,...,...,...
30497,2023-10-25,127.066891,37.535869,0,영업/정상,서울시광진구휴게음식점인허가정보.csv
30498,2023-10-25,127.061296,37.536253,0,영업/정상,서울시광진구휴게음식점인허가정보.csv
30499,2023-10-27,127.085362,37.557365,0,영업/정상,서울시광진구휴게음식점인허가정보.csv
30500,2023-11-02,127.078977,37.555599,0,영업/정상,서울시광진구휴게음식점인허가정보.csv


In [9]:
#!pip install geopandas

In [11]:
test.reset_index(inplace=True,drop=True)
import geopandas as gpd
from shapely.geometry import Point, Polygon
# Geopandas 지오데이터프레임 생성
gdf = gpd.GeoDataFrame(test, geometry=gpd.points_from_xy(test['좌표정보(X)'], test['좌표정보(Y)']))

# 격자 생성
xmin, ymin, xmax, ymax = [gdf.total_bounds[i] for i in range(4)]
width = height = 0.0089 # 대략 1km
rows = int((ymax-ymin) / height)
cols = int((xmax-xmin) / width)
grid = []
for i in range(cols):
    for j in range(rows):
        grid.append(Polygon([
            (xmin + i * width, ymin + j * height),
            (xmin + (i + 1) * width, ymin + j * height),
            (xmin + (i + 1) * width, ymin + (j + 1) * height),
            (xmin + i * width, ymin + (j + 1) * height)
        ]))

# 격자별로 데이터 포인트 매핑
grid_gdf = gpd.GeoDataFrame(geometry=grid)
joined = gpd.sjoin(gdf, grid_gdf, how='left', op='within')

# 격자별 데이터 집계
aggregated_data = joined.groupby('index_right').size()

# 결과 출력 혹은 저장

C:\Users\HP\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [12]:
joined.to_csv('격자화_인프라.csv',index=False)
# index_right 열에  고유한 격자 정보 저장.
# 아직 index_right에는 유의미한 정보는 없으므로 해당 정보만으로 어떤 격자인지 확인이 불가능함
# 따라서 해당 격자가 잘 군집화 되었는지 시각화 해서 확인할 필요가 있음.

---

## 생성한 격자가 데이터를 잘 군집하는지 시각화를 통해 확인 *임시
- 일자 별 데이터 프레임을 별도로 만들 예정이며, 본 코드는 시각화를 통해 군집화 상태를 확인하기 위함임.

## 격자 별 인프라 정보 삽입

In [13]:
test_grid = pd.DataFrame(joined[['정보','index_right']].groupby(by='index_right').value_counts())
# 격자정보(index_right) 별 인프라 정보의 합

In [14]:
test_grid = test_grid.unstack()
test_grid = test_grid.fillna(0)
# 인프라가 nan인 것은 없다는 것으로 해석할 수 있으므로 0으로 변환

In [15]:
test_grid

0                                          \
정보          서울시광진구골프연습장업인허가정보.csv 서울시광진구공연장인허가정보.csv 서울시광진구관광숙박업인허가정보.csv   
index_right                                                                 
0.0                           0.0                0.0                  0.0   
114287.0                      0.0                0.0                  0.0   
114288.0                      0.0                0.0                  1.0   
114746.0                      2.0                1.0                  0.0   
114747.0                     10.0                5.0                  1.0   
114748.0                      0.0                3.0                  4.0   
114749.0                      0.0                0.0                  0.0   
115205.0                      2.0                1.0                  0.0   
115206.0                      1.0                0.0                  1.0   
115207.0                      0.0                3.0                  0.0   
115208.0                      8.0                1.0                  2.0   
115664.0                      7.0                1.0                  1.0   
115665.0                      9.0                0.0                  1.0   
115666.0                      2.0                1.0                  0.0   
115667.0                      3.0                0.0                  0.0   
116123.0                      1.0               11.0                  0.0   
116124.0                     16.0                0.0                  0.0   
116125.0                      2.0                0.0                  0.0   
116126.0                      1.0                0.0                  0.0   
116583.0                      0.0                0.0                  0.0   
116584.0                      3.0                2.0                  3.0   
116585.0                      0.0                0.0                  0.0   

                                                                             \
정보          서울시광진구노래연습장업인허가정보.csv 서울시광진구단란주점영업인허가정보.csv 서울시광진구당구장업인허가정보.csv   
index_right                                                                   
0.0                           0.0                   0.0                 0.0   
114287.0                     10.0                   2.0                 8.0   
114288.0                     16.0                   8.0                23.0   
114746.0                      7.0                   0.0                 3.0   
114747.0                    116.0                  51.0                76.0   
114748.0                     42.0                  56.0                55.0   
114749.0                      2.0                   2.0                 5.0   
115205.0                     41.0                  16.0                38.0   
115206.0                      2.0                   3.0                15.0   
115207.0                      4.0                   1.0                11.0   
115208.0                     53.0                  32.0                55.0   
115664.0                     19.0                  13.0                19.0   
115665.0                     66.0                  75.0                54.0   
115666.0                     23.0                  10.0                23.0   
115667.0                     24.0                  18.0                46.0   
116123.0                      2.0                   0.0                 4.0   
116124.0                     15.0                   9.0                 8.0   
116125.0                      0.0                   0.0                 1.0   
116126.0                      2.0                   0.0                 0.0   
116583.0                      1.0                   0.0                 0.0   
116584.0                      6.0                  11.0                10.0   
116585.0                      0.0                   0.0                 0.0   

                                                                          \
정보          서울시광진구대규모점포인허가정보.csv 서울시광진구동물병원인허가정보.csv 서

In [16]:
import random

def generate_unique_hex_colors(num_colors):
    colors = set()
    
    while len(colors) < num_colors:
        color = "#{:06x}".format(random.randint(0, 0xFFFFFF))
        colors.add(color)
    
    return list(colors)

unique_hex_colors = generate_unique_hex_colors(22)

# 고유한 격자 총 22개에 매핑할 헥사 코드 생성

In [17]:
arr = list(test_grid.index)
obj = {}
for i in range( len(arr)):
    obj[arr[i]]=unique_hex_colors[i]
# 고유한 격자 코드(obj)에 생성한 헥사 코드 매핑

In [18]:
ts = joined.set_index('index_right')
ts['color'] = obj
# 시각화를 위한 새로운 데이터 프레임 생성 및 color 열을 추가.
# color 열에 앞서 생성한 헥사 코드 삽입

In [19]:
m = folium.Map(location=[37.555, 127.081], zoom_start=13)
# 지도 생성

In [20]:
a = []
for i in range( ts.shape[0]):
    arr =[ts.iloc[i][1],ts.iloc[i][2],ts.iloc[i][-1]]
    a.append(arr)
# 지도에 찍을 좌표와 색상 코드 리스트화

In [21]:
def set_folium(x,y,z):
    folium.CircleMarker(
        location=[x, y],
        radius=5,
        color=z,  # z는 사용자 지정 헥사 코드
        fill=True,
        fill_color=z
    ).add_to(m)
for i in a[::100]:
    set_folium(i[1],i[0],i[2])
# 지도에 찍기

In [22]:
m
# 시각화

- 각 적절한 군집화(약 1km^2)가 이뤄졌음을 확인할 수 있음.
---

## 2. 광진구 내 불법 주정차 단속 위치별 건수 격자화
- 시각화는 진행하지 않음
- 일자 별로 정리 필요

In [23]:
illigal_df = pd.read_csv('광진구_위경도_단속.csv')

In [24]:
illigal_df['정보'] = '단속장소'

In [25]:
illigal_df

,단속일시,과태료,단속지역,단속장소,위반내용,견인지시,단속구분,위도,경도,정보
0,2020-03-17,32000,자양동,서울 광진구 자양동 662-5,횡단보도,미견인,스마트앱(서울시),37.532973,127.072741,단속장소
1,2020-03-20,40000,자양동,서울 광진구 자양동 662-5,횡단보도,미견인,스마트앱(서울시),37.532973,127.072741,단속장소
2,2020-03-23,32000,자양동,서울 광진구 자양동 662-5,횡단보도,미견인,스마트앱(서울시),37.532973,127.072741,단속장소
3,2020-03-23,40000,자양동,서울 광진구 자양동 662-5,횡단보도,미견인,스마트앱(서울시),37.532973,127.072741,단속장소
4,2020-03-29,40000,자양동,서울 광진구 자양동 662-5,횡단보도,미견인,스마트앱(서울시),37.532973,127.072741,단속장소
...,...,...,...,...,...,...,...,...,...,...
134761,2023-03-23 16:30:00,32000,중곡동,하이프라자 광진군자점 인근,주정차금지(황색실선)구역,미견인,고정형CCTV,37.556815,127.080881,단속장소
134762,2023-03-24 16:56:00,40000,중곡동,하이프라자 광진군자점 인근,주정차금지(황색실선)구역,미견인,고정형CCTV,37.556815,127.080881,단속장소
134763,2023-03-29 07:14:00,40000,중곡동,하이프라자 광진군자점 인근,주정차금지(황색실선)구역,미견인,고정형CCTV,37.556815,127.080881,단속장소
134764,2023-03-31 14:48:00,32000,중곡동,하이프라자 광진군자점 인근,주정차금지(황색실선)구역,미견인,고정형CCTV,37.556815,127.080881,단속장소


In [26]:
gdf_2 = gpd.GeoDataFrame(illigal_df, geometry=gpd.points_from_xy(illigal_df['경도'], illigal_df['위도']))

# 격자별로 데이터 포인트 매핑
grid_gdf = gpd.GeoDataFrame(geometry=grid)
joined = gpd.sjoin(gdf_2, grid_gdf, how='left', op='within')
# 기존 격자 기준에 매핑
# 격자별 데이터 집계
aggregated_data = joined.groupby('index_right').size()

# 결과 출력 혹은 저장

C:\Users\HP\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [27]:
joined.to_csv('격자화_불법주정차단속.csv',index=False)

In [28]:
test_grid_2 = pd.DataFrame(joined[['정보','index_right']].groupby(by='index_right').value_counts())
# 격자정보(index_right) 별 인프라 정보의 합

In [29]:
test_grid_2 = test_grid_2.unstack()
test_grid_2 = test_grid_2.fillna(0)
# nan인 것은 없다는 것으로 해석할 수 있으므로 0으로 변환

In [30]:
test_meta_df = pd.concat([test_grid,test_grid_2],axis=1).fillna(0)
# 인프라 + 불법 주정차 단속 관련 데이터 프레임 concat

## 3. 광진구 내 불법 주정차 단속 cctv 위치 격자화

In [31]:
cctv_df = pd.read_csv('수정일자_이전_설치CCTV.csv')

In [32]:
cctv_df['정보'] = 'cctv위치'

In [33]:
cctv_df

,고정형CCTV지번주소,위도,경도,자치구,단속지점명,현장구분,등록일시,단속지번주소,정보
0,서울 광진구 광장동 334-2,37.547790,127.106990,광진구,SK강평주유소 앞,불법주정차구역,2020-03-17 14:02:57,서울 광진구 광장동 334-2,cctv위치
1,서울 광진구 광장동 445-3,37.543799,127.102410,광진구,광현교회 앞,불법주정차구역,2020-03-17 14:02:57,서울 광진구 광장동 445-3,cctv위치
2,서울 광진구 구의동 135,37.549593,127.081780,광진구,아차산역 5번출구 앞,불법주정차구역,2020-03-17 14:02:57,서울 광진구 구의동 135,cctv위치
3,서울 광진구 군자동 503,37.548609,127.071072,광진구,두산위브 앞,불법주정차구역,2020-03-17 14:02:57,서울 광진구 군자동 503,cctv위치
4,서울 광진구 구의동 66-62,37.545348,127.088420,광진구,국민은행 앞,불법주정차구역,2020-03-17 14:02:57,서울 광진구 구의동 66-62,cctv위치
5,서울 광진구 구의동 68-29,37.545976,127.086327,광진구,구의사거리,불법주정차구역,2020-03-17 14:02:57,서울 광진구 구의동 68-29,cctv위치
6,서울 광진구 구의동 546-1,37.533558,127.093389,광진구,리젠트오피스텔건너편,불법주정차구역,2020-03-17 14:02:57,서울 광진구 구의동 546-1,cctv위치
7,서울 광진구 구의동 546-11,37.535562,127.093359,광진구,지너스타워 주차장 앞,불법주정차구역,2020-03-17 14:02:57,서울 광진구 구의동 546-11,cctv위치
8,서울 광진구 구의동 611,37.538347,127.096620,광진구,현대2단지아파트 앞,불법주정차구역,2020-03-17 14:02:57,서울 광진구 구의동 611,cctv위치
9,서울 광진구 구의동 546-6,37.535668,127.094522,광진구,강변역 1번 출구,불법주정차구역,2020-03-17 14:02:57,서울 광진구 구의동 546-6,cctv위치


In [34]:
gdf_3 = gpd.GeoDataFrame(cctv_df, geometry=gpd.points_from_xy(cctv_df['경도'], cctv_df['위도']))

# 격자별로 데이터 포인트 매핑
grid_gdf = gpd.GeoDataFrame(geometry=grid)
joined = gpd.sjoin(gdf_3, grid_gdf, how='left', op='within')
# 기존 격자 기준에 매핑
# 격자별 데이터 집계
aggregated_data = joined.groupby('index_right').size()

# 결과 출력 혹은 저장

C:\Users\HP\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [35]:
joined.to_csv('격자화_cctv장소.csv',index=False)

In [36]:
test_grid_3 = pd.DataFrame(joined[['정보','index_right']].groupby(by='index_right').value_counts())
# 격자정보(index_right) 별 인프라 정보의 합

In [37]:
test_grid_3 = test_grid_3.unstack()
test_grid_3 = test_grid_3.fillna(0)
# nan인 것은 없다는 것으로 해석할 수 있으므로 0으로 변환

In [38]:
test_meta_df = pd.concat([test_meta_df,test_grid_3],axis=1).fillna(0)
# 인프라 + 불법 주정차 단속 관련 데이터 프레임 + cctv 개수

## 수정이후 cctv

In [39]:
test_gird_plus =pd.read_csv('수정일자_이후_설치CCTV.csv')

In [40]:
gdf_plus = gpd.GeoDataFrame(test_gird_plus, geometry=gpd.points_from_xy(test_gird_plus['경도'], test_gird_plus['위도']))

# 격자별로 데이터 포인트 매핑
grid_gdf = gpd.GeoDataFrame(geometry=grid)
joined = gpd.sjoin(gdf_plus, grid_gdf, how='left', op='within')
# 기존 격자 기준에 매핑
# 격자별 데이터 집계
aggregated_data = joined.groupby('index_right').size()

# 결과 출력 혹은 저장

C:\Users\HP\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [41]:
joined.to_csv('격자화_수정_이후_csv',index=False)

## 4. 최종 격자 메타 데이터 프레임 생성

In [42]:
test_meta_df

0                                          \
정보          서울시광진구골프연습장업인허가정보.csv 서울시광진구공연장인허가정보.csv 서울시광진구관광숙박업인허가정보.csv   
index_right                                                                 
0.0                           0.0                0.0                  0.0   
114287.0                      0.0                0.0                  0.0   
114288.0                      0.0                0.0                  1.0   
114746.0                      2.0                1.0                  0.0   
114747.0                     10.0                5.0                  1.0   
114748.0                      0.0                3.0                  4.0   
114749.0                      0.0                0.0                  0.0   
115205.0                      2.0                1.0                  0.0   
115206.0                      1.0                0.0                  1.0   
115207.0                      0.0                3.0                  0.0   
115208.0                      8.0                1.0                  2.0   
115664.0                      7.0                1.0                  1.0   
115665.0                      9.0                0.0                  1.0   
115666.0                      2.0                1.0                  0.0   
115667.0                      3.0                0.0                  0.0   
116123.0                      1.0               11.0                  0.0   
116124.0                     16.0                0.0                  0.0   
116125.0                      2.0                0.0                  0.0   
116126.0                      1.0                0.0                  0.0   
116583.0                      0.0                0.0                  0.0   
116584.0                      3.0                2.0                  3.0   
116585.0                      0.0                0.0                  0.0   
105102.0                      0.0                0.0                  0.0   
114286.0                      0.0                0.0                  0.0   

                                                                             \
정보          서울시광진구노래연습장업인허가정보.csv 서울시광진구단란주점영업인허가정보.csv 서울시광진구당구장업인허가정보.csv   
index_right                                                                   
0.0                           0.0                   0.0                 0.0   
114287.0                     10.0                   2.0                 8.0   
114288.0                     16.0                   8.0                23.0   
114746.0                      7.0                   0.0                 3.0   
114747.0                    116.0                  51.0                76.0   
114748.0                     42.0                  56.0                55.0   
114749.0                      2.0                   2.0                 5.0   
115205.0                     41.0                  16.0                38.0   
115206.0                      2.0                   3.0                15.0   
115207.0                      4.0                   1.0                11.0   
115208.0                     53.0                  32.0                55.0   
115664.0                     19.0                  13.0                19.0   
115665.0                     66.0                  75.0                54.0   
115666.0                     23.0                  10.0                23.0   
115667.0                     24.0                  18.0                46.0   
116123.0                      2.0                   0.0                 4.0   
116124.0                     15.0                   9.0                 8.0   
116125.0                      0.0                   0.0                 1.0   
116126.0                      2.0                   0.0                 0.0   
116583.0                      1.0                   0.0                 0.0   
116584.0                      6.0                  11.0                10.0   
116585.0                      0.0                   0.0 

### setting_grid 파일을 통해 데이터 분석 계획 상 구상했던 데이터 프레임 형태를 만들 수 있음을 확인함
- 앞서 제작한 test_meta_df 의 형태로 각 격자 별 데이터를 기간 별로 구분하여, 별도의 데이터 프레임을 생성할 것임.

# 이후 계획

#### 생성할 데이터 프레임 목록
1. 2022년 1분기 이전 데이터 프레임
2. 2022년 2분기 ~ 2022년 4분기 까지 데이터 프레임
3. 2023년 전체 데이터 프레임

#### 모델 학습 계획
- 학습 및 평가에는 1,2 데이터 프레임을 활용한다.
- 1,2 데이터 프레임을 통해 cctv의 추가 설치 및 인프라 데이터의 변화에 따른 불법 주정차 단속 건수의 영향을 학습시킨다.
- 3 데이터 프레임을 통해 최종 모델의 성능을 테스트 하고, 임의로 cctv 개수에 변화를 주어, 설치 당 가장 큰 효율을 제공하는 cctv 설치 위치 (격자)를 결과로써 제공한다.

